# MOSQUIMAP
## Mapping urban malaria vector habitat suitability and human population exposure

<div id="toc"></div>

#Javascript code for building ToC

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Setting the working environment

**Importing Python libraries and modules**

In [2]:
# Libraries for setting parameters of operating system 
import os
import sys
# Pandas, Numpy, Scipy libraries
import pandas as pd
import numpy as np
import scipy as scipy
# Module for finding pathnames matching a pattern (Unix style)
import glob

**Adding directory containing Python scripts that complement this notebook**

In [3]:
# Add 'scripts' directory to the path (directory 'scripts' contains python scripts with custom functions)
src = os.path.abspath('../scripts')
if src not in sys.path:
    sys.path.append(src)

**Setting up environment variables**

Please edit the script `../scripts/config_windows.py` or `../scripts/config_linux.py`containing the configuration parameters, according to your own computer setup. The following cell runs this script.

In [4]:
run ../scripts/config_windows.py

In [5]:
# Import functions that setup the environment variables
import environ_variables_windows as envi

# Set environment variables
envi.setup_environment_variables() 
# Display current environment variables of your computer
# Uncomment in case of issue, it can help find what is going wrong
#envi.print_environment_variables()

**Importing GRASS GIS package and module**

In [6]:
#Import the GRASS GIS Python scripting package
import grass.script as gscript

#Import the GRASS GIS module containing setup, initialization and clean-up functions
import grass.script.setup as gsetup


**Importing custom functions (from scripts in the 'scripts' directory)**

In [7]:
# Function that checks if GRASS GIS database directories exist, and creates them if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset, launch_mapset

# Function that checks if GRASS GIS add-on is installed and installs it if needed
# Due to problems with g.extension in scripts, it is probably better to install addons using the GRASS GUI
from gextension import check_install_addon

# Function for time management
from processing_time import start_processing, print_processing_time

# Function that checks if a directory exists and creates it if needed
from mkdirectory import check_create_dir



**Creating directories if they do not exist**

Please make sure all the required input data are present in the 'indata' directories. Pairwise comparison matrices should be stored as csv files.

In [8]:
#Check and create directories
check_create_dir(indata['geodata'])
check_create_dir(indata['matrices_larvae'])
check_create_dir(indata['matrices_adults'])
check_create_dir(outdata['outputdir'])
check_create_dir(outdata['output_matrices_analysis'])
check_create_dir(outdata['output_vif_analysis'])
check_create_dir(outdata['output_validation'])
check_create_dir(rules['rules_dir'])

The directory 'D:\Sabine\mosquimap_workflow\input_geodata' already exists
The directory 'D:\Sabine\mosquimap_workflow\input_matrices_larvae' already exists
The directory 'D:\Sabine\mosquimap_workflow\input_matrices_adults' already exists
The directory 'D:\Sabine\mosquimap_workflow\output_geodata' already exists
The directory 'D:\Sabine\mosquimap_workflow\output_matrices_analysis' already exists
The directory 'D:\Sabine\mosquimap_workflow\output_vif_analysis' already exists
The directory 'D:\Sabine\mosquimap_workflow\output_validation_data' already exists
The directory 'D:\Sabine\mosquimap_workflow\rules' already exists


In [9]:
# Check paths where Python searches for system modules
# Uncomment in case of issue, it can help find what is going wrong
#import sys
#sys.path

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# GRASS location and mapset

In [10]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'DAKAR_32628' already exist",
 "'PERMANENT' mapset already exists in location 'DAKAR_32628'",
 "You are now working in mapset 'DAKAR_32628/PERMANENT'"]

# GRASS addons

In [11]:
# Check if add-on is already installed and install it if needed
check_install_addon("r.geomorphon")

r.geomorphon is already installed on your computer


In [12]:
# Check if add-on is already installed and install it if needed
check_install_addon("r.vif")

r.vif is already installed on your computer


In [13]:
# Check if add-on is already installed and install it if needed
check_install_addon("r.texture.tiled")

r.texture.tiled is already installed on your computer


In [14]:
# Check if add-on is already installed and install it if needed
check_install_addon("v.concave.hull")

v.concave.hull is already installed on your computer


# Import input data

In [15]:
# Start timer
starttime_importdata=start_processing()

## Link the satellite image

In [17]:
#Link Pleiades image (the file is not imported, a link is created)
#Only band 4 is necessary for the processing
#If you don't need to view the other bands and you want to speed up processing, link only band 4
gscript.run_command('r.external', input=indata['image'], output='PLEIADES')

## Mask out areas that are outside AOI

In [16]:
# Import the AOI polygon and use it as a mask to limit processing to the AOI
# The AOI polygon covers Dakar, Guediawaye, Pikine and some urban communes of Rufisque
gscript.run_command('v.in.ogr', overwrite=True, input=indata['aoi'], output='AOI')
gscript.run_command('r.mask', overwrite=True, vector='AOI')


0

## Import layers

In [17]:
#Import AOI for validation (smaller extent, where survey was carried out)
#This is the area where the entomological surveys were carried out
#It will be used as computational region for validation
gscript.run_command('v.in.ogr', overwrite=True, input=indata['aoi4validation'], output='AOI_VAL')

#Import AOI for running tests (smaller extent, set the region to this AOI 
# for faster processing when trying some function)
gscript.run_command('v.in.ogr', overwrite=True, input=indata['aoi4tests'], output='AOI_SMALL')

#Import Pleiades land cover at 0.5m resol 
#From MAUPP/REACT, with correction of obvious misclassification of airport, visual interpretation of landfill,
#and waterbodies split into 3 classes according to size
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=0.5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['landcover'], 
                    output='LAND_COVER_ORIG')

#Import Pleiades land use at 0.5m resol(from MAUPP/REACT, corrected for topology errors)
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=0.5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['landuse'], 
                    output='LU')

#Import Pleiades DTM resampled to 5m resol, and mask of DTM failed pixels (produced in Geomatica)
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['dtm_5m'],
                    output='DTM')
gscript.run_command('r.in.gdal', overwrite=True, input=indata['dtm_5m_maskfailed'],
                    output='DTM_FAILED')

#Import soil pH at 30m resol (iSDAsoil - Hengl et al., 2021)
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['soil_ph_30m'],
                    output='SOIL_PH_TEMP')

#Import Topographic Wetness Index produced in SAGA at 5m resol
#SAGA TWI is better than GRASS TWI
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['twi'],
                    output='TWI_TEMP')

#Import population distribution modelled in MAUPP at 100m
gscript.run_command('g.region', vector='AOI', align='DTM', res=100)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['pop'] ,output='POP')

#Import streams (vector layer from OSM and local knowledge)
gscript.run_command('v.in.ogr', overwrite=True, input=indata['streams'], output='STREAMS_OSM')

#Import marine waters (vector layer from OSM and local knowledge)
gscript.run_command('v.in.ogr', overwrite=True, input=indata['marine_waters'], output='MARINE_OSM')

#Import the binary lc class "waterbodies" (output from REACT)
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=0.5)
gscript.run_command('r.in.gdal', overwrite=True, input=indata['lc_waterbodies'], 
                   output='WATERBODIES')

#Import the anopheles presence points, for "winter 2013" (rainy season)
#The points were sampled by UCAD in the field
gscript.run_command('v.in.ogr', overwrite=True, input=indata['larvae_presence'], output='ANOPHELES')


0

In [18]:
## Print processing time
print_processing_time(starttime_importdata ,"Data imported in ")

'Data imported in 3 minutes and 9.9 seconds'

# Gridded population density map (elements at risk)


In [19]:
# Start timer
starttime_pop=start_processing()

#Set computational region
gscript.run_command('g.region', raster='POP', res=100)



0

## Log-transforming and scaling the population values

In [20]:
#Input population map is already gridded, no need to aggregate

#Log-transform population density
gscript.run_command('r.mapcalc', overwrite=True, expression="POP1 = POP + 1 ") #1 is added to avoid errors due to 0 values
gscript.run_command('r.mapcalc', overwrite=True, expression="POP_LOG = log(POP1)")

#Scale the values to [0;100] 
rastinfo = gscript.raster_info('POP_LOG')
outmax = '%s_max' %('POP_LOG')
rastmax = rastinfo['max']
print(outmax, '=', rastmax ) 
outmin = '%s_min' %('POP_LOG')
rastmin = rastinfo['min']
print(outmin, '=', rastmin )

formula = '%s = round(((%s - %s) / (%s - %s)*100))' %('POP_LOG','POP_LOG',rastmin,rastmax,rastmin)
print(formula)
gscript.mapcalc(formula, overwrite=True)

POP_LOG_max = 6.89306155644823
POP_LOG_min = 0.0
POP_LOG = round(((POP_LOG - 0.0) / (6.89306155644823 - 0.0)*100))


## Binning

In [21]:
#Calculate percentiles
gscript.run_command('r.quantile', overwrite=True, input='POP_LOG', percentiles= [50, 80], flags='r', file = rules['recode_pop2classes'])
#Recode according to percentiles
gscript.run_command('r.recode', overwrite=True, input= 'POP_LOG', output='POP_LOG_CLASSES', rules= rules['recode_pop2classes'])

0

## Exporting

In [22]:
#Export map to tiff file, for preparing map layout in QGIS (if needed)
gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='POP_LOG_CLASSES', output= os.path.join(outdata['outputdir'],'dakar_human_population_classes_100m.tif'))
#gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='POP_LOG', output=  os.path.join(outdata['outputdir'],'dakar_human_population_scaled_100m.tif'))


0

In [23]:
## Print processing time
print_processing_time(starttime_pop ,"Gridded population density map created in ")

'Gridded population density map created in 2.1 seconds'

# Gridded larval habitat suitability map


## Producing criteria

In [24]:
# Start timer
starttime_criteria_larvae=start_processing()

**8 layers (factor groups) are used in the larvae habitat suitability analysis**<br />
Land cover (categorical)<br />
Land use (categorical)<br />
Landforms (categorical) or relative elevation (continuous)<br />
Soil moisture (continuous)<br />
Soil pH (continuous)<br />
Distance to buildings (continuous)<br />
Distance to trees (continuous)<br />
Water pollution (continuous)

### Land cover (categorical)

The original LC layer that was imported has the following 12 classes:
1 Low-rise buildings, 2 Swimming pools, 3 Paved surface, 4 Dumpsites, 5 Bare soil, 6 Low vegetation, 7 Trees, 8 Small waterbodies, 9 Medium waterbodies, 10 Large waterbodies, 11 Shadow, 12 Medium- and high-rise buildings <br> <br>
The LC layer used in the larvae analysis has the following 14 classes:
1 Buildings, 2 Swimming pools, 3 Paved surface, 4 Dumpsites, 5 Bare soil, 6 Grass, 7 Shrubs, 8 Trees, 9 Small water bodies, 10 Medium water bodies, 11 Large water bodies, 12 Water courses, 13 Marine waters, 14 Shadow <br> 

From the original LC layer, classes Low rise buildings and Medium- and High-rise buildings must be merged into a single class Buildings <br> 
Class Low vegetation must be split into classes Grass and Shrubs <br>
Classes Water courses and Marine waters must be added

In [25]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=0.5)

0

#### Assigning a suitable colour palette to the original LC layer

In [26]:
#Assign a suitable LC colour palette (for viewing, illustrations...)
gscript.run_command('r.colors', map='LAND_COVER_ORIG', rules= rules['colors_lc_12cl'])

0

#### Merging buidings into a single class

In [27]:
### Merge building classes 1 and 12 into class 1
formula = 'LAND_COVER_TEMP = (if(LAND_COVER_ORIG == 12, 1, LAND_COVER_ORIG))'
gscript.mapcalc(formula, overwrite=True)


#### Splitting waterbodies into sub-classes according to their size

In [28]:
#Vectorize the waterbodies
gscript.run_command('r.to.vect', overwrite=True,input='WATERBODIES', output='WATERBODIES', type = 'area')
gscript.run_command('v.edit', overwrite=True, map='WATERBODIES', tool = 'delete', where= 'value = 0' )

#Select REACT waterbodies that overlap OSM streams and convert them to raster
gscript.run_command('v.edit', overwrite=True, map='WATERBODIES', tool = 'delete', where= 'value = 0' )
gscript.run_command('v.select', overwrite=True, ainput='WATERBODIES', binput = 'STREAMS_OSM', output = 'STREAMS_LC', operator = "overlap")
#Convert them to raster and set NULL to 0
gscript.run_command('v.to.rast', overwrite=True, input='STREAMS_LC', output = 'STREAMS_LC', use= 'val', value = "100")
gscript.run_command('r.null', overwrite=True, map='STREAMS_LC', null=0)

#Select REACT waterbodies that overlap OSM marine waters and convert them to raster
gscript.run_command('v.select', overwrite=True, ainput='WATERBODIES', binput = 'MARINE_OSM', output = 'MARINE_LC', operator = "overlap")
#Convert them to raster and set NULL to 0
gscript.run_command('v.to.rast', overwrite=True, input='MARINE_LC', output = 'MARINE_LC', use= 'val', value = "200")
gscript.run_command('r.null', overwrite=True, map='MARINE_LC', null=0)

#Sum the LC and the newly created layers where streams have a value between 100 and 199
#and marine waters have a value between 200 and 299
formula = 'LAND_COVER_TEMP2 =(LAND_COVER_TEMP + STREAMS_LC + MARINE_LC)'
gscript.mapcalc(formula, overwrite=True)


#### Splitting low vegetation into Grass and Shrubs

<span style="color:blue">Warning: GLCM-based textures are computer-intensive. Expect a long processing time for the next cell.</span>

In [32]:
#Compute GLCM homogeneity (aka Inverse Difference Moment - IDM) of the NIR band of the Pleiades image, 
#in all directions, in an 11x11 window, with a lag of 1.
#There was an error on 'intl-8.dll', so I renamed the file 'old_intl-8.dll' (C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\Library\bin\old_intl-8.dll)
#In the same directory, there is a file named intl-8_2.dll
gscript.run_command('r.texture.tiled', overwrite=True, input='PLEIADES.4', output='NIR', 
                    size=11, method='idm', distance='1', tile_width='1000', tile_height='1000', processes='12')

#Reclass the homogeneity layer based on a threshold (for differenciating grass and shrubs)
#lower than or equal to 0.18 is 300 (shrubs)
#higher than 0.18 is 0 (grass)
#The threshold was set based on visual interpretation
formula = 'NIR_IDM_THR = (if(NIR_IDM<=0.18, 300, 0))'
gscript.mapcalc(formula, overwrite=True)

#Replace class 6 Low vegetation with 2 classes : Grass (0) and Shrubs (300)
formula = 'LAND_COVER_TEMP3 = (if(LAND_COVER_TEMP2==6, NIR_IDM_THR, LAND_COVER_TEMP2))'
gscript.mapcalc(formula, overwrite=True)


#Assign a random colour palette (for viewing)
gscript.run_command('r.colors', map='LAND_COVER_TEMP3', color='random')

0

#### Recoding the new LC map (number the 14 classes from 1 to 14)

In [29]:
#Recode the new LC map
gscript.run_command('r.recode', overwrite=True, input='LAND_COVER_TEMP3', 
                    output='LC', rules= rules['recode_lc'])

#Assign a suitable LC colour palette (for viewing, illustrations...)
gscript.run_command('r.colors', map='LC', rules=rules['colors_lc_14cl'])

0

### Land use (categorical)

In [30]:
#Land use layer is used as such
#No preprocessing required

### Landforms (categorical) and relative elevation (continuous)

In [31]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [32]:
#Produce landforms and calculate mean relative elevation based on the DTM
gscript.run_command('r.geomorphon', overwrite=True, elevation='DTM', intensity='REL_ELEVATION_TEMP', 
                    forms='LANDFORMS_TEMP', search=60, skip=0, flat=0.3, dist=0)

0

In [33]:
#Correct artefacts in LANDFORMS_TEMP due to failed pixels in DTM
#Failed pixels occur in waterbodies, so landforms value can be set to 9 (valleys) where they occur
formula = 'LANDFORMS = (if(DTM_FAILED==0, 9, LANDFORMS_TEMP))'
gscript.mapcalc(formula, overwrite=True)


In [34]:
#Correct artefacts in REL_ELEVATION_TEMP due to failed pixels in DTM
#Failed pixels occur in waterbodies, so mean relative elevation value can be set to min value where they occur
formula = 'REL_ELEVATION = (if(DTM_FAILED==0, -1, REL_ELEVATION_TEMP))'
gscript.mapcalc(formula, overwrite=True)


### Soil moisture: Topographic Wetness Index - TWI (continuous)

In [35]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [36]:
#Correct artefacts in TWI (computed in SAGA GIS) due to failed pixels in DTM
#Failed pixels occur in waterbodies, so TWI value can be set to max where they occur
rastinfo=gscript.raster_info('TWI_TEMP')
rastmax = rastinfo['max']
print(rastmax)
formula = 'TWI = (if(DTM_FAILED==0, %s, TWI_TEMP))' %(rastmax)
gscript.mapcalc(formula, overwrite=True)


21.35587


### Soil pH (continuous)

In [37]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [38]:
##Fill holes in the pH layer

#Set values 255 and 0 to null
gscript.run_command('r.null', overwrite=True, map='SOIL_PH_TEMP', setnull='255,0')

#Fill holes with rst method, and round the values
gscript.run_command('r.fillnulls', overwrite=True, input='SOIL_PH_TEMP', output='SOIL_PH_TEMP2', method='rst')
gscript.run_command('r.mapcalc', overwrite=True,expression="SOIL_PH = round( SOIL_PH_TEMP2 )")

#Assign a suitable colour palette (for viewing, illustrations...)
gscript.run_command('r.colors', map = 'soil_ph', color = 'bgyr')

0

### Distance to buildings (continuous)

In [39]:
#Set computational region
gscript.run_command('g.region', raster='DTM', res=5)

0

In [40]:
#Reclassify the land cover to keep only the buildings, calculate the distance from the buildings (in m)
gscript.run_command('r.reclass', overwrite=True, input='LC', output='BUILDINGS_TEMP', 
                    rules=rules['reclass_lc2buildings'])
gscript.run_command('r.grow.distance', overwrite=True, input='BUILDINGS_TEMP', 
                    distance='DIST_BUILDINGS', flags = 'm')


0

### Distance to trees (continuous)

In [41]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [42]:
#Reclassify the land cover to keep only the trees, calculate the distance from the trees (in m)
gscript.run_command('r.reclass', overwrite=True, input='LC', 
                    output='TREES_TEMP', rules=rules['reclass_lc2trees'])
gscript.run_command('r.grow.distance', overwrite=True, input='TREES_TEMP', distance='DIST_TREES', flags = 'm')


0

### Distance to dumpsites - water pollution (continuous)

In [43]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [44]:
#Reclassify the land cover to keep only the dumpsites, calculate the distance from the dumpsites (in m)
gscript.run_command('r.reclass', overwrite=True, input='LC', 
                    output='DUMPSITES_TEMP', rules=rules['reclass_lc2dumpsites'])
gscript.run_command('r.grow.distance', overwrite=True, input='DUMPSITES_TEMP', 
                    distance='DIST_DUMPSITES', flags = 'm')


0

In [45]:
## Print processing time
print_processing_time(starttime_criteria_larvae ,"Criteria created in ")

'Criteria created in 16 minutes and 24.3 seconds'

## Assessing multicollinearity of criteria with VIF

In [46]:
# Start timer
starttime_multicol=start_processing()

In [47]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [48]:
gscript.run_command('r.vif', overwrite=True, maps= ['LC', 'LU', 'REL_ELEVATION', 'TWI', 'SOIL_PH', 'DIST_BUILDINGS', 'DIST_TREES', 'DIST_DUMPSITES'], file= os.path.join(outdata['output_vif_analysis'],'VIF_8layers.txt'))

0

In [49]:
## Print processing time
print_processing_time(starttime_multicol ,"Multicollinearity assessed in ")

'Multicollinearity assessed in 3 minutes and 16.5 seconds'

####  <span style="color:purple">STOP ! Please check the console or the 'VIF_8layers.txt' file to ensure criteria are not correlated.</span>

In [53]:
input('Press enter to continue')

Press enter to continue 


' '

## Scaling factors

In [50]:
# Start timer
starttime_scaling_factors=start_processing()

### AHP for scaling categorical factors (land cover, land use, land forms)

In [51]:
### Compute (1) factor weights (priority vector) and (2) consistency ratio of each pairwise matrix

##COMPARE THE RELATIVE SUITABILITY OF SUBFACTORS FOR ASSIGNING A WEIGHT TO EACH OF THEM
##ASSESS THE CONSISTENCY OF THE JUDGMENTS

#Comment on the warnings (from documentation): 
#When the matrix for which the eigenvalues and eigenvectors are computed
#is real, the resulting eigenvalues are real (0 imaginary part) or occur in conjugate pairs

#Create a list of csv files with the pairwise comparison matrices (PCM), and count the csv files
file_list = glob.glob(os.path.join(indata['matrices_larvae_intra'], '*.csv'))
print("There are %s paths in the list"%len(file_list))

#Loop over the list of csv files
for file in file_list:
    #Read the pairwise comparison matrix (csv with headers)
    pcm_matrix = pd.read_csv(file, sep=';')   
    #print (pcm_matrix)

    #Convert to array
    pcm = np.array(pcm_matrix)
    #print (pcm)

    ##1-Calculate the priority vector (weight vector) of each PCM, i.e., its principal eigenvector
    
    eigenvalues, eigenvector=np.linalg.eig(pcm)
    maxindex=np.argmax(eigenvalues)
    eigenvalues=np.float32(eigenvalues) #float32
    eigenvector=np.float32(eigenvector) #float32
    weights=eigenvector[:, maxindex]

    weights = weights/np.sum(weights)

    print ("Layer Weights: " + str(np.around(weights,4)))
        
    #Create a csv file with the weights
    filename = 'weight_vector_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    np.savetxt(path_file, weights, fmt='%1.4f')
    
    ##2-Calculate the consistency ratio (CR) of each PCM, 
    ##i.e., its consistency index (CI) versus the consistency index of a random-like array (RI) of same size
    ##CR=CI/RI
    ##CR<0.10 is acceptable (Saaty, 2012)

    ##2.1-Calculate the Consistency Index (CI)

    #Multiply the pairwise comparison matrix by the weights, in column
    pcm_mult = np.multiply(pcm, weights)
    #print ("Matrix multiplied by weights (in column) =", pcm_mult)

    #Sum the lines
    weighted_sum = np.sum(pcm_mult, axis=1)
    #print ("Weighted sum (in line) =", weighted_sum)

    #Mean of (weighted sums divided by weights (priority))
    lambda_max = np.average(np.divide(weighted_sum, weights))
    #print("Lambda max =", lambda_max)

    #Count number of rows in pairwise comparison matrix
    nb_rows = pcm.shape[0]
    #print("Number of rows in the matrix =", nb_rows)

    #Consistency Index (Lambda max - nb rows)/(nb_rows-1)
    ci = ((lambda_max-nb_rows)/(nb_rows-1))
    #print("Consistency Index =", ci)
    
    ##2.2-Random Index (RI)

    #The RI values, as a function of the matrix size, are given in (Saaty et al., 2007)
    ri = 0.0
    if nb_rows == 3:
        ri = 0.52
    elif nb_rows == 4:
        ri = 0.89
    elif nb_rows == 5:
        ri = 1.11
    elif nb_rows == 6:
        ri = 1.25
    elif nb_rows == 7:
        ri = 1.35
    elif nb_rows == 8:
        ri = 1.40
    elif nb_rows == 9:
        ri = 1.45
    elif nb_rows == 10:
        ri = 1.49
    elif nb_rows == 11:
        ri = 1.52
    elif nb_rows == 12:
        ri = 1.54
    elif nb_rows == 13:
        ri = 1.56
    elif nb_rows == 14:
        ri = 1.58
    elif nb_rows == 15:
        ri = 1.59
    #print("Random Index (RI) =", ri)

    ##2.3-Calculate Consistency Ratio (CR)
    cr = (ci/ri)
    #print("Consistency Ratio (CR) =",cr)
    
    #Create a csv file with the consistency index, the random index, and the consistency ratio
    filename = 'consistency_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    ciricr = (ci,ri,cr)
    np.savetxt(path_file, ciricr)
 

There are 15 paths in the list
Layer Weights: [0.0136 0.016  0.0343 0.0573 0.1197 0.0573 0.0499 0.0522 0.1661 0.1999
 0.0742 0.028  0.0417 0.0899]
Layer Weights: [0.0104 0.053  0.0337 0.0268 0.0189 0.0457 0.0309 0.0264 0.122  0.1853
 0.2609 0.1002 0.0652 0.0206]


C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:28: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part


Layer Weights: [0.0164 0.025  0.0184 0.0115 0.0456 0.0312 0.0312 0.014  0.2625 0.2115
 0.1363 0.0768 0.0988 0.0209]
Layer Weights: [0.0213 0.0489 0.0453 0.0473 0.0427 0.0766 0.0532 0.0264 0.289  0.1678
 0.0932 0.0156 0.022  0.0507]
Layer Weights: [0.0368 0.0358 0.0187 0.0842 0.0648 0.072  0.0345 0.0347 0.2695 0.179
 0.1119 0.026  0.0097 0.0224]
Layer Weights: [0.138  0.0249 0.0297 0.0518 0.06   0.0274 0.2649 0.2523 0.085  0.0661]
Layer Weights: [0.0658 0.013  0.0165 0.0539 0.0292 0.0304 0.3319 0.2429 0.144  0.0724]
Layer Weights: [0.0161 0.0149 0.0186 0.0283 0.0485 0.0244 0.1594 0.15   0.2293 0.3104]
Layer Weights: [0.0868 0.0297 0.0306 0.0334 0.0334 0.0323 0.323  0.1889 0.1328 0.1093]
Layer Weights: [0.0684 0.0273 0.0281 0.0281 0.0242 0.0167 0.2862 0.1458 0.1991 0.1761]
Layer Weights: [0.0261 0.1375 0.066  0.1498 0.0363 0.1126 0.1033 0.3684]
Layer Weights: [0.0176 0.0781 0.1025 0.2228 0.0388 0.1542 0.0623 0.3237]
Layer Weights: [0.0146 0.2381 0.043  0.186  0.0535 0.0297 0.1644 0.2706]

In [52]:
#### Compute the geometric mean of weights

from scipy.stats import gmean

def readmycsv(args):
    return pd.read_csv(args, header=None)

##Create a list per factor
land_cover = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_larvae_lc*.csv'))
land_use = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_larvae_lu*.csv'))
land_forms = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_larvae_lf*.csv'))

##Land cover
#Concatenate csv files and replace 0 by 1 (to avoid error when computing geometric mean)
weights_concat = pd.concat(map(readmycsv, land_cover), axis = 1, ignore_index=True)
weights_concat =  weights_concat.replace(0,1)
#Compute geometric mean, scale it and export to csv file
weights_gmean = weights_concat.apply(gmean, axis=1)
weights_gmean = ((weights_gmean - weights_gmean.min()) / (weights_gmean.max() - weights_gmean.min()))*100
weights_gmean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lc_geomean.csv'), header=None,index=False, float_format="%.0f")

##Land use
#Concatenate csv files and replace 0 by 1 (to avoid error when computing geometric mean)
weights_concat = pd.concat(map(readmycsv, land_use), axis = 1, ignore_index=True)
weights_concat =  weights_concat.replace(0,1)
#Compute geometric mean, scale it and export to csv file
weights_gmean = weights_concat.apply(gmean, axis=1)
weights_gmean = ((weights_gmean - weights_gmean.min()) / (weights_gmean.max() - weights_gmean.min()))*100
weights_gmean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lu_geomean.csv'), header=None,index=False, float_format="%.0f")

##Land forms
#Concatenate csv files and replace 0 by 1 (to avoid error when computing geometric mean)
weights_concat = pd.concat(map(readmycsv, land_forms), axis = 1, ignore_index=True)
weights_concat =  weights_concat.replace(0,1)
#Compute geometric mean, scale it and export to csv file
weights_gmean = weights_concat.apply(gmean, axis=1)
weights_gmean = ((weights_gmean - weights_gmean.min()) / (weights_gmean.max() - weights_gmean.min()))*100
weights_gmean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lf_geomean.csv'), header=None,index=False, float_format="%.0f")

In [53]:
#### Produce the categorical factor layers

##LAND COVER

#Create a dataframe (df) with the recoding rules
df_larvae_lc_weights = pd.read_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lc_geomean.csv'), sep=';', header=None)
df_larvae_lc_weights['lc_class'] = range(1, len(df_larvae_lc_weights) + 1)
df_larvae_lc_weights['equal'] = '='
df_larvae_lc_weights = df_larvae_lc_weights[['lc_class', 'equal', 0]]
#print (df_larvae_lc_weights)


#Create a CSV file with the recoding rules (no header, no index, separator is space)
df_larvae_lc_weights.to_csv(rules['classes_larvae_lc'], header=False, index=False, sep = ' ')

#Reclassify the predictor layer
gscript.run_command('r.reclass', overwrite=True, input='LC', output='L_LC', rules=rules['classes_larvae_lc'])


##LAND USE

#Create a dataframe (df) with the recoding rules
df_larvae_lu_weights = pd.read_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lu_geomean.csv'), sep=';', header=None)
df_larvae_lu_weights['lu_class'] = range(1, len(df_larvae_lu_weights) + 1)
df_larvae_lu_weights['equal'] = '='
df_larvae_lu_weights = df_larvae_lu_weights[['lu_class', 'equal', 0]]
#print (df_larvae_lu_weights)


#Create a CSV file with the recoding rules (no header, no index, separator is space)
df_larvae_lu_weights.to_csv(rules['classes_larvae_lu'], header=False, index=False, sep = ' ')

#Reclassify the predictor layer
gscript.run_command('r.reclass', overwrite=True, input='LU', output='L_LU', rules=rules['classes_larvae_lu'])


##LAND FORMS

#Create a dataframe (df) with the recoding rules
df_larvae_lf_weights = pd.read_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_lf_geomean.csv'), sep=';', header=None)
df_larvae_lf_weights['lf_class'] = range(1, len(df_larvae_lf_weights) + 1)
df_larvae_lf_weights['equal'] = '='
df_larvae_lf_weights = df_larvae_lf_weights[['lf_class', 'equal', 0]]
#print (df_larvae_lf_weights)


#Create a CSV file with the recoding rules (no header, no index, separator is space)
df_larvae_lf_weights.to_csv(rules['classes_larvae_lf'], header=False, index=False, sep = ' ')

#Reclassify the predictor layer
gscript.run_command('r.reclass', overwrite=True, input='LANDFORMS', output='L_LF', rules=rules['classes_larvae_lf'])


0

### Membership functions for scaling continuous factors

In [54]:
#Set computational region (use DTM geometry DTM, 5m*5m)
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [55]:
##TOPOGRAPHIC WETNESS

#Rescale the topographic wetness index (TWI) to [0;100]
gscript.run_command('r.rescale', overwrite=True, input= 'TWI', output = 'L_TW', to=[0,100])

0

In [56]:
##SOIL pH

#Rescale the soil pH to [0;100]
gscript.run_command('r.rescale', overwrite=True, input= 'SOIL_PH', output = 'L_PH', to=[0,100])

0

In [57]:
##DISTANCE TO BUILDINGS


#At a distance of 0m, we are in the buildings so suitability is 0
#Between 0 and 10m, suitability is maximal (100)
#Between 10 and 100m, suitability decreases with distance
#After 100m, suitability is 0 again


#Calculate the slope and intercept of the linear function
#for rescaling the distance interval [10;100] to [50;0]
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress([10,100],[50,0])
print(slope,intercept)

#Calculate the factor value as a linear function of the distance, as y = ax + b (valid only for interval [10,100])
formula = '%s = (%s * %s) + %s' %('TEMP',slope,'DIST_BUILDINGS',intercept)
gscript.mapcalc(formula, overwrite=True)

#Calculate the factor value
formula = "L_BU = round(if (DIST_BUILDINGS == 0, 0, if (DIST_BUILDINGS < 10, 100, if (DIST_BUILDINGS >= 10 && DIST_BUILDINGS < 100, TEMP, 0))))"
gscript.mapcalc(formula, overwrite=True)

-0.5555555555555556 55.55555555555556


In [58]:
##DISTANCE TO TREES

#At a distance of 0m, we are in the trees so suitability is 0
#Between 0 and 300m, suitability decreases with distance
#After 300m, the suitability is 0 again

#Calculate the slope and intercept of the linear function
#for rescaling the distance interval [1,100] to [100, 0]
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress([1,100],[100,0])
print(slope,intercept)

#Calculate the factor value as a linear function of the distance, as y = ax + b (valid only for interval [1,100])
formula = '%s = (%s * %s) + %s' %('TEMP',slope,'DIST_TREES',intercept)
gscript.mapcalc(formula, overwrite=True)

#Calculate the factor value
formula = "L_TR = round(if (DIST_TREES == 0, 0, if (DIST_TREES >= 1 && DIST_TREES < 100, TEMP, 0)))"
gscript.mapcalc(formula, overwrite=True)

-1.0101010101010102 101.01010101010101


In [59]:
##DISTANCE TO DUMPSITES/LANDFILLS (WATER POLLUTION)

#At a distance of 0m, we are in the dumpsite and suitability is not affected
#Up to about 350m of Mbeubeuss, suitability is low due to pollution
#From 350m to 750m, suitability increases with distance
#After 750m, suitability is not affected


#Calculate the slope and intercept of the linear function
#for rescaling the distance interval [350,750] to [0, 100]
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress([350,750],[0,100])
print(slope,intercept)

#Calculate the factor value as a linear function of the distance, as y = ax + b (valid only for interval [1,100])
formula = '%s = (%s * %s) + %s' %('TEMP',slope,'DIST_DUMPSITES',intercept)
gscript.mapcalc(formula, overwrite=True)

#Calculate the factor value
formula = "L_DU = round(if (DIST_DUMPSITES == 0, 100, if (DIST_DUMPSITES >= 1 && DIST_DUMPSITES < 350, 0, if (DIST_DUMPSITES >= 350 && DIST_DUMPSITES < 750, TEMP, 100))))"
gscript.mapcalc(formula, overwrite=True)

0.25 -87.5


In [60]:
## Print processing time
print_processing_time(starttime_scaling_factors ,"Factors scaled in ")

'Factors scaled in 22.9 seconds'

## Weighting factors

In [61]:
##COMPARE THE RELATIVE IMPORTANCE OF THE FACTORS, AND THE ASSESS THE CONSISTENCY OF THE JUDGMENTS

#Comment on the warnings (from documentation): 
#When the matrix for which the eigenvalues and eigenvectors are computed
#is real, the resulting eigenvalues are real (0 imaginary part) or occur in conjugate pairs

#Create a list of csv files that contain the pairwise comparison matrices (PCM), and count the csv files
file_list = glob.glob(os.path.join(indata['matrices_larvae_inter'], '*.csv'))
print("There are %s paths in the list"%len(file_list))

#Loop over the list of csv files
for file in file_list:
    #Read the pairwise comparison matrix (csv with headers) into df
    pcm_matrix = pd.read_csv(file, sep=';')   
    #print (pcm_matrix)

    #Convert into array
    pcm = np.array(pcm_matrix)
    #print (pcm)

    ##1-Calculate the priority vector (weight vector) of each PCM, i.e., its principal eigenvector
    
    eigenvalues, eigenvector=np.linalg.eig(pcm)
    maxindex=np.argmax(eigenvalues)
    eigenvalues=np.float32(eigenvalues) #float32
    eigenvector=np.float32(eigenvector) #float32
    weights=eigenvector[:, maxindex]

    weights = weights/np.sum(weights)

    #print ("Layer Weights: " + str(np.around(weights,4)))
    
    
    #Create a csv file with the weights
    filename = 'weight_vector_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    np.savetxt(path_file, weights, fmt='%1.4f')
    
    ##2-Calculate the consistency ratio (CR) of each PCM, 
    ##i.e., its consistency index (CI) versus the consistency index of a random-like array (RI) of same size
    ##CR=CI/RI
    ##CR<0.10 is acceptable (Saaty, 2012)

    ##2.1-Consistency Index (CI)

    #Multiply the pairwise comparison matrix by the weights, in column
    pcm_mult = np.multiply(pcm, weights)
    #print ("Matrix multiplied by weights (in column) =", pcm_mult)

    #Sum the lines
    weighted_sum = np.sum(pcm_mult, axis=1)
    #print ("Weighted sum (in line) =", weighted_sum)

    #Mean of (weighted sums divided by weights (priority))
    lambda_max = np.average(np.divide(weighted_sum, weights))
    #print("Lambda max =", lambda_max)

    #Count number of rows in pairwise comparison matrix
    nb_rows = pcm.shape[0]
    #print("Number of rows in the matrix =", nb_rows)

    #Consistency Index (Lambda max - nb rows)/(nb_rows-1)
    ci = ((lambda_max-nb_rows)/(nb_rows-1))
    #print("Consistency Index =", ci)
    
    ##2.2-Random Index (RI)

    #The RI values, as a function of the matrix size, are given in (Saaty et al., 2007)
    ri = 0.0
    if nb_rows == 3:
        ri = 0.52
    elif nb_rows == 4:
        ri = 0.89
    elif nb_rows == 5:
        ri = 1.11
    elif nb_rows == 6:
        ri = 1.25
    elif nb_rows == 7:
        ri = 1.35
    elif nb_rows == 8:
        ri = 1.40
    elif nb_rows == 9:
        ri = 1.45
    elif nb_rows == 10:
        ri = 1.49
    elif nb_rows == 11:
        ri = 1.52
    elif nb_rows == 12:
        ri = 1.54
    elif nb_rows == 13:
        ri = 1.56
    elif nb_rows == 14:
        ri = 1.58
    elif nb_rows == 15:
        ri = 1.59
    #print("Random Index (RI) =", ri)

    ##2.3-Consistency Ratio (CR)
    cr = (ci/ri)
    #print("Consistency Ratio (CR) =",cr)
    
    #Create a csv file with the consistency index, the random index, and the consistency ratio
    filename = 'consistency_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    ciricr = (ci,ri,cr)
    np.savetxt(path_file, ciricr)
    

There are 5 paths in the list


C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:26: ComplexWarning: Casting complex values to real discards the imaginary part


In [62]:
#### Compute arithmetic mean of weights

def readmycsv(args):
    return pd.read_csv(args, header=None)

##Create list of csv files
factors = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_larvae_factors*.csv'))

#Concatenate csv files
weights_concat = pd.concat(map(readmycsv, factors), axis = 1, ignore_index=True)
#Compute arithmentic mean and export to csv file
weights_mean = weights_concat.mean(axis=1)
weights_mean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_larvae_factors_mean.csv'), header=None,index=False, float_format="%.4f")


In [63]:
#Create summary CSVs with consistency index, random index, and consistency ratio

##Create lists
land_cover = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_larvae_lc*.csv'))
land_use = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_larvae_lu*.csv'))
land_forms = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_larvae_lf*.csv'))
factors = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_larvae_factors*.csv'))

#Concatenate land cover consistency
consistency_concat = pd.concat(map(readmycsv, land_cover), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'],"consistency_larvae_concat_lc.csv"), index=False, header=None, float_format="%.4f")

#Concatenate land use consistency
#consistency_concat = pd.concat(map(readmycsv, land_use), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'], "consistency_larvae_concat_lu.csv"), index=False, header=None, float_format="%.4f")

#Concatenate land forms consistency
#consistency_concat = pd.concat(map(readmycsv, land_forms), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'], "consistency_larvae_concat_lf.csv"), index=False, header=None, float_format="%.4f")

#Concatenate factors consistency
#consistency_concat = pd.concat(map(readmycsv, factors), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'], "consistency_larvae_concat_factors.csv"), index=False, header=None, float_format="%.4f")


####  <span style="color:purple">STOP ! Please check the consistency ratio and evaluate if experts should revise their judgments.</span>

In [53]:
input('Press enter to continue')

Press enter to continue 


' '

## Aggregating criteria to produce the larval HSI map

In [64]:
# Start timer
starttime_hsi=start_processing()

In [65]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

0

In [66]:
#Calculate the weighted sum of factors to create the layer "LARVAE_SUITABILITY_SC1"

df_larvae_factors_weights = pd.read_csv(os.path.join(outdata['output_matrices_analysis'], 'weight_vectors_larvae_factors_mean.csv'), header=None)
print(df_larvae_factors_weights)
w1 = df_larvae_factors_weights[0].values[0]
w2 = df_larvae_factors_weights[0].values[1]
w3 = df_larvae_factors_weights[0].values[2]
w4 = df_larvae_factors_weights[0].values[3]
w5 = df_larvae_factors_weights[0].values[4]
w6 = df_larvae_factors_weights[0].values[5]
w7 = df_larvae_factors_weights[0].values[6]
w8 = df_larvae_factors_weights[0].values[7]

formula = '%s = (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s)' %('LARVAE_SUITABILITY_SC1',w1, 'L_LC', w2, 'L_LU', w3, 'L_LF', w4, 'L_TW', w5, 'L_PH', w6, 'L_BU', w7, 'L_TR', w8, 'L_DU')
gscript.mapcalc(formula, overwrite=True)

#Scale the values to [0;100] 
rastinfo = gscript.raster_info('LARVAE_SUITABILITY_SC1')
outmax = '%s_max' %('LARVAE_SUITABILITY_SC1')
rastmax = rastinfo['max']
print(outmax, '=', rastmax ) 
outmin = '%s_min' %('LARVAE_SUITABILITY_SC1')
rastmin = rastinfo['min']
print(outmin, '=', rastmin )

formula = '%s = round(((%s - %s) / (%s - %s)*100))' %('LARVAE_SUITABILITY_SC1','LARVAE_SUITABILITY_SC1',rastmin,rastmax,rastmin)
print(formula)
gscript.mapcalc(formula, overwrite=True)

        0
0  0.0680
1  0.1239
2  0.1386
3  0.2466
4  0.1240
5  0.0513
6  0.0419
7  0.2057
LARVAE_SUITABILITY_SC1_max = 86.0734
LARVAE_SUITABILITY_SC1_min = 14.9368
LARVAE_SUITABILITY_SC1 = round(((LARVAE_SUITABILITY_SC1 - 14.9368) / (86.0734 - 14.9368)*100))


In [67]:
#Combine with constraints


#Use a 50m buffer inside AOI polygon to mask out beaches and rocks near seashore
gscript.run_command('v.buffer', overwrite=True, input= "AOI", output="AOI_BUFFER50", type="area", distance=-50)
gscript.run_command('v.to.rast', overwrite=True, input='AOI_BUFFER50', output = 'AOI_BUFFER50', use= 'val', value = "1")
gscript.run_command('r.null', overwrite=True, map='AOI_BUFFER50', null=0)

#Use LC classes and buffer as constraints (unsuitable areas= buildings, paved soil, trees, streams, marine waters, buffer)
gscript.run_command('r.mapcalc', overwrite=True, 
                    expression="LARVAE_SUITABILITY_SC1_CONSTR = if (LC == 1 ||  LC == 3 || LC == 8 ||  LC == 12 ||  LC == 13, 0, if(AOI_BUFFER50 == 0,0, LARVAE_SUITABILITY_SC1))")


0

In [68]:
#Export the HSI map
gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='LARVAE_SUITABILITY_SC1_CONSTR', 
                    output=os.path.join(outdata['outputdir'],'dakar_larvae_hsi_5m_full_aoi.tif'))


In [69]:
## Print processing time
print_processing_time(starttime_hsi,"HSI map created in ")

'HSI map created in 14.4 seconds'

## Validation (case where survey data are available)

### Validation : Aggregating HSI (i) in grids and (ii) in buffers around presence points (case where survey data are available)

In [70]:
# Start timer
starttime_aggrhsi=start_processing()

In [71]:
#Set computational region (use DTM geometry, 5m*5m)
gscript.run_command('g.region', vector='AOI', align='DTM', res= 5)

0

#### Calculating HSI in fishnet and in buffers around presence points, export as CSV files

Create a regular grid (fishnet) that fully covers the AOI, and calculate statistics in grid cells  
Besides, compute the same statistics around each positive observation point (buffer same size as grid cells)  
Export the values to 2 csv files(fishnet and buffers) for computing Continuous Boyce Index (CBI)
and plotting HSI vs P/E in R.

Repeat for multiple grid sizes is possible (e.g., 15m, 25m, 45m, 95m,...)


In [72]:
#Create list (if you created more than one map, add them to the list)
raster_list = ['LARVAE_SUITABILITY_SC1_CONSTR']

#Loop over cell sizes expressed in meters (if you wish to validate in multiple cell sizes, complete the list)
#(e.g. n = np.array([15,25,35,95]))
n = np.array([15])

for x in np.nditer(n):
    
      
    ##GRID##

    #Generate a grid with cells of x meters * x meters (fishnet)
    tempgrid1='GRID_'+str(x)+'_TEMP1'
    gscript.run_command('v.mkgrid', overwrite=True, map=tempgrid1, box=[x,x])

    #Drop the unnecessary columns 'row' and 'col'
    gscript.run_command('v.db.dropcolumn', map=tempgrid1, columns='row')
    gscript.run_command('v.db.dropcolumn', map=tempgrid1, columns='col')

    #Select only (i) gridcells that fall completely within the validation AOI, AND
    #(ii) gridcells that contain presence points but do not fall completely within the validation AOI
    antemp='ANOPHELES_'+str(x)+'_TEMP'
    tempgrid2='GRID_'+str(x)+'_TEMP2'
    tempgrid3='GRID_'+str(x)+'_TEMP3'
    tempgrid4='GRID_'+str(x)+'_TEMP4'
    grid='GRID_'+str(x)
    
    gscript.run_command('v.select', overwrite=True, ainput=tempgrid1, binput='AOI_VAL', 
                        operator='within',output= tempgrid2 )
    gscript.run_command('v.select', overwrite=True, ainput='ANOPHELES', binput=tempgrid2, 
                        operator='disjoint',output=antemp)
    gscript.run_command('v.select', overwrite=True, ainput=tempgrid1, binput=antemp, 
                        operator='contains',output= tempgrid3)
    gscript.run_command('v.patch', overwrite=True, flags='e', input=[tempgrid2,tempgrid3], 
                        output=tempgrid4)
    gscript.run_command('v.clean', overwrite=True, input=tempgrid4, output=grid, 
                        tool=['rmdupl','rmdac'])


    

    ##MANAGE ATTRIBUTES##
    
    #Drop the column 'presence', if it has been added to ANOPHELES during a previous run 
    gscript.run_command('v.db.dropcolumn', map='ANOPHELES', columns='presence')
    #Add a column 'presence' to ANOPHELES and populate it with value 1
    gscript.run_command('v.db.addcolumn', map='ANOPHELES', columns='presence integer')
    gscript.run_command('v.db.update', map='ANOPHELES', column='presence', value='1')

    
    #Drop the columns that were added to the grid during the previous run 
    gscript.run_command('v.db.dropcolumn', map=grid, 
                        columns=['presence','suitability_average'])
    #Add a column 'presence' to the grid and populate it with value 1 from ANOPHELES
    #for grid cells where a there is a presence point
    gscript.run_command('v.db.addcolumn', map=grid, columns='presence integer')
    gscript.run_command('v.what.vect', map=grid, column='presence', query_map='ANOPHELES', 
                        query_column='presence')
    #Replace NULL values with 0 in column 'presence'
    gscript.run_command('v.db.update', map=grid, column='presence', value=0, 
                        where='presence IS NULL')
    
    


    ##CALCULATE STATISTICS##
    
 
    for raster in raster_list:

        #Calculate suitability statistics in the grid (average)
        #The computational region is AOI to avoid null values in cells not completely within AOI_VAL.
        gscript.run_command('g.region', vector='AOI', align='DTM', res=5)
        gscript.run_command('v.rast.stats', map=grid, raster=raster, flags='c',
                            column_prefix='suitability', method='average')
        gridstats='GRID_%s_'%(raster)+str(x)
        gscript.run_command('g.copy',overwrite=True,vector=[grid,gridstats])
               

        #Calculate suitability statistics in the buffers around presence points
        neigh= 'NEIGH_%s'%(raster)+str(x)
        y=int(x/5) #y must be odd
        buffstats='BUFFERS_%s_'%(raster)+str(x)
        gscript.run_command('r.neighbors', overwrite=True, input=raster, output=neigh, 
                            method='average', size=y)
        gscript.run_command('v.what.rast', overwrite=True, map='ANOPHELES', type='point', 
                            raster=raster, column='suitability_average')
        gscript.run_command('g.copy',overwrite=True,vector=['ANOPHELES',buffstats])
        gscript.run_command('v.db.dropcolumn', map='ANOPHELES', columns=['suitability_average'])
       
        
        

        ##EXPORT CSV files##

        #Export the grid's attribute table as a csv file
        gridcsv= os.path.join(outdata['output_validation'],'larvae_hsi_%s_'%(raster[19:])+str(x)+'.csv')
        gscript.run_command('db.out.ogr', overwrite=True, input=gridstats, output=gridcsv)
        #print(gridcsv)

        #Export the buffers' attribute table as a csv file
        buffcsv=os.path.join(outdata['output_validation'],'larvae_hsi_presence_%s_'%(raster[19:])+str(x)+'.csv')
        gscript.run_command('db.out.ogr', overwrite=True, input=buffstats, output=buffcsv)
        #print(buffcsv)
        

In [73]:
## Print processing time
print_processing_time(starttime_aggrhsi,"HSI aggregated in ")

'HSI aggregated in 35 minutes and 49.2 seconds'

### Validation : Calculating CBI and plotting HSI vs P/E (R script)

####  <span style="color:purple">STOP ! Please run the following cell in R (e.g., RStudio) and then proceed with processing in this notebook, or use R magic to call R code from this notebook.</span>
#### <span style="color:purple">Class breaks should be defined in Excel following Hirzel's method before proceeding. They should be stored in a txt file in the 'rules' directory. </span>
Hirzel, A.H., Le Lay, G., Helfer, V., Randin, C., Guisan, A., 2006. Evaluating the ability of habitat suitability models to predict species presences. Ecological Modelling, Predicting Species Distributions 199, 142–152. https://doi.org/10.1016/j.ecolmodel.2006.05.017


In [80]:
input('Press enter to continue')

Press enter to continue 


' '

#########R script#########

#Call the ecospat library
library(ecospat)

#Set the input working directory according to your settings
setwd("   ")

#Import the csv file with hsi and presence data, as a dataframe
#Note that unique() removes duplicate rows due to overlapping buffers in GRASS
larvae_hsi <- read.csv(file = 'larvae_hsi_SC1_15.csv')
larvae_hsi_presence <- unique(read.csv(file = 'larvae_hsi_presence_SC1_15.csv'))

#Count the missing values (there should not be missing value)
sum(is.na(larvae_hsi))
sum(is.na(larvae_hsi_presence))

#Add a column with a sequential number starting from 1
larvae_hsi$num<-1:nrow(larvae_hsi)
larvae_hsi_presence$num<-1:nrow(larvae_hsi_presence)


#Check the type of each column in the dataframe
sapply(larvae_hsi, class)
sapply(larvae_hsi_presence, class)

#Prepare the variables for calculating the Continuous Boyce Index (CBI)
#The argument fit is a vector containing the predicted suitability values
#The argument obs is a vector containing the predicted suitability values of the validation points (presence records)
fit <-  larvae_hsi$suitability_average
obs <- larvae_hsi_presence$suitability_average


#Calculate and plot the Continuous Boyce Index (CBI)
larvae_cbi_SC1_CONSTR_15_average <- ecospat.boyce(fit, obs, nclass = 0, window.w = 'default', res = 100, PEplot = TRUE)
larvae_cbi_SC1_CONSTR_15_average$Spearman.cor

#Set the output working directory according to your settings
setwd("")

#Save the plot and optionally close the graphics device
dev.print(svg, 'larvae_cbi_SC1_CONSTR_15_average.svg')
#dev.off() #uncomment to close the graphics device

#Round to 3 decimal digits (useful, e.g., to for use in Excel)
round_larvae_cbi_SC1_CONSTR_15_average <- lapply(larvae_cbi_SC1_CONSTR_15_average,round,digits=3)

#Print CBI and F-value (= maximum F-ratio )
CBI <- round_larvae_cbi_SC1_CONSTR_15_average$Spearman.cor
Fvalue <- max(round_larvae_cbi_SC1_CONSTR_15_average$F.ratio)

print(paste("CBI = ",CBI))
print(paste("F-value =", Fvalue))

#Export as csv to the 'output_validation_data' directory (set path according to your settings)
write.csv(round_larvae_cbi_SC1_CONSTR_15_average, file = "..\\larvae_cbi_SC1_CONSTR_15_average.csv", row.names = FALSE)
                                                        

## Binning (e.g., 4 classes: Unsuitable, Marginal, Suitable and Optimal)

In [74]:
# Start timer
starttime_catmap=start_processing()

In [75]:
#Set computational region (use DTM geometry, 5m*5m)
gscript.run_command('g.region', vector='AOI', align='DTM', res= 5)


0

In [76]:
#Calculate aggregated HSI (average) in a 3x3 pixel neighbourhood (15m). Resolution is still 5m but values are smoothed.
#Reclassify HSI average based on HSI vs F-ratio curve (in our case, upper class limits: 43.0, 50.0, 75.0, 100.0)
gscript.run_command('r.neighbors', overwrite=True, input='LARVAE_SUITABILITY_SC1_CONSTR', 
                    output='NEIGH_LARVAE_SUITABILITY_SC1_CONSTR', method='average', size=3)
gscript.run_command('r.recode', overwrite=True, input= 'NEIGH_LARVAE_SUITABILITY_SC1_CONSTR', 
                    output='LARVAE_HABITAT_SUITABILITY_CLASSES', rules=rules['recode_larvalhsi2classes'])

0

#Use this cell instead if the aim is to produce a layer that is resampled to 15m
gscript.run_command('g.region', res= 15)

gscript.run_command('r.resamp.stats', overwrite=True, input='LARVAE_SUITABILITY_SC1_CONSTR', 
                    output='NEIGH_LARVAE_SUITABILITY_SC1_CONSTR', method='average')
gscript.run_command('r.recode', overwrite=True, input= 'NEIGH_LARVAE_SUITABILITY_SC1_CONSTR', 
                    output='LARVAE_HABITAT_SUITABILITY_CLASSES', rules=rules['recode_larvalhsi2classes'])

In [77]:
#Exporting the map to tiff file, for making map layout in QGIS
gscript.run_command('r.out.gdal', overwrite=True, flags= 'm',input='LARVAE_HABITAT_SUITABILITY_CLASSES', 
                    output=os.path.join(outdata['outputdir'],'dakar_larvae_hs_classes_5m.tif'))

0

In [78]:
#Mask areas outside the population map
gscript.run_command('r.mask', overwrite=True, raster='POP')

#Export the HSI map with the restrictive mask (not beyond the population layer extent)
#gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='LARVAE_SUITABILITY_SC1_CONSTR', 
#                    output=os.path.join(outdata['outputdir'],'dakar_larvae_hsi_5m.tif'))

0

In [79]:
## Print processing time
print_processing_time(starttime_catmap,"Map created in ")

'Map created in 5.9 seconds'

# Gridded adult Anopheles habitat suitability maps (hazard)

## Producing criteria: land cover, land use, distance to breeding sites, distance to buildings

In [80]:
# Start timer
starttime_criteria=start_processing()

**Land cover**  

The LC layer that was imported has the following 12 classes:
1 Low-rise buildings, 2 Swimming pools, 3 Paved surface, 4 Dumpsites, 5 Bare soil, 6 Low vegetation, 7 Trees, 8 Small waterbodies, 9 Medium waterbodies, 10 Large waterbodies, 11 Shadow, 12 Medium- and high-rise buildings <br> <br>
The LC layer used in the adult vector analysis has the following 10 classes:
1 Low-rise buildings, 2 Medium- and high-rise buildings, 3 Swimming pools, 4 Paved surface, 5 Dumpsites, 6 Bare soil, 7 Grass, 8 Trees and shrubs, 9 Water bodies, 10 Shadow <br> 

Classes 1 and 12 must be merged in a single class buildings <br> 
Low vegetation must be split into Grass and Shrubs <br>
Waterbodies must be split into isolate Water courses, Marine waters

In [81]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='PLEIADES.1', res=0.5)

##LAND COVER

#Merge all water bodies from the original LC in a single class
#Order the classes as in the matrices filled out by experts
#The resulting LC has 10 classes
gscript.run_command('r.recode', overwrite=True, input='LAND_COVER_ORIG', output='LC_A_TEMP', rules=rules['recode_lca'])

#To reclass grass, trees, shrubs, use the LC layer created for larvae suitability where they are distinct
gscript.run_command('r.mapcalc',overwrite=True, expression='LC_A = int(if (LC == 7,8,LC_A_TEMP))')

#Assign a suitable LC colour palette (for viewing, illustrations...)
gscript.run_command('r.colors', map='LC_A', rules=rules['colors_lc_10cl'])

0

In [82]:
##LAND USE

#The LU classes are the same for larvae and adult suitability
#No additional pre-processing required

In [83]:
#Set computational region (use DTM geometry, 5m*5m)
gscript.run_command('g.region', vector='AOI', align='DTM', res= 5)

##DISTANCE TO BREEDING SITES

#Make a binary raster from the larval habitat suitability classes
#Class 4 (optimal) = 1
#Other classes = 0
#This can be adapted to the context and the season
gscript.run_command('r.mapcalc', overwrite=True, expression='L_HS_BINARY = if (LARVAE_HABITAT_SUITABILITY_CLASSES == 4, 1, 0)')
gscript.run_command('r.null', overwrite=True, map = 'L_HS_BINARY', setnull = '0')

#Calculate a distance raster from the suitable classes
gscript.run_command('r.grow.distance', overwrite=True, flags='m', input='L_HS_BINARY', distance='L_HS_DIST')

0

In [84]:
## Print processing time
print_processing_time(starttime_criteria,"Criteria produced in ")

'Criteria produced in 4 minutes and 22.6 seconds'

## Scaling factors

In [85]:
# Start timer
starttime_scaling_factors=start_processing()

In [86]:
#Set computational region (use DTM geometry, 5m*5m)
gscript.run_command('g.region', vector='AOI', align='DTM', res= 5)

0

### AHP for scaling categorical factors (land cover, land use)

In [87]:
### Compute (1) factor weights (priority vector) and (2) consistency ratio of each pairwise matrix

##COMPARE THE RELATIVE SUITABILITY OF SUBFACTORS FOR ASSIGNING A WEIGHT TO EACH OF THEM
##ASSESS THE CONSISTENCY OF THE JUDGMENTS

#Comment on the warnings (from documentation): 
#When the matrix for which the eigenvalues and eigenvectors are computed
#is real, the resulting eigenvalues are real (0 imaginary part) or occur in conjugate pairs

#Create a list of csv files with the pairwise comparison matrices (PCM), and count the csv files
file_list = glob.glob(os.path.join(indata['matrices_adults_intra'], '*.csv'))
print("There are %s paths in the list"%len(file_list))

#Loop over the list of csv files
for file in file_list:
    #Read the pairwise comparison matrix (csv with headers)
    pcm_matrix = pd.read_csv(file, sep=';')   
    #print (pcm_matrix)

    #Convert to array
    pcm = np.array(pcm_matrix)
    #print (pcm)

    ##1-Calculate the priority vector (weight vector) of each PCM, i.e., its principal eigenvector
    
    eigenvalues, eigenvector=np.linalg.eig(pcm)
    maxindex=np.argmax(eigenvalues)
    eigenvalues=np.float32(eigenvalues) #float32
    eigenvector=np.float32(eigenvector) #float32
    weights=eigenvector[:, maxindex]

    weights = weights/np.sum(weights)

    print ("Layer Weights: " + str(np.around(weights,4)))
        
    #Create a csv file with the weights
    filename = 'weight_vector_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    np.savetxt(path_file, weights, fmt='%1.4f')
    
    ##2-Calculate the consistency ratio (CR) of each PCM, 
    ##i.e., its consistency index (CI) versus the consistency index of a random-like array (RI) of same size
    ##CR=CI/RI
    ##CR<0.10 is acceptable (Saaty, 2012)

    ##2.1-Calculate the Consistency Index (CI)

    #Multiply the pairwise comparison matrix by the weights, in column
    pcm_mult = np.multiply(pcm, weights)
    #print ("Matrix multiplied by weights (in column) =", pcm_mult)

    #Sum the lines
    weighted_sum = np.sum(pcm_mult, axis=1)
    #print ("Weighted sum (in line) =", weighted_sum)

    #Mean of (weighted sums divided by weights (priority))
    lambda_max = np.average(np.divide(weighted_sum, weights))
    #print("Lambda max =", lambda_max)

    #Count number of rows in pairwise comparison matrix
    nb_rows = pcm.shape[0]
    #print("Number of rows in the matrix =", nb_rows)

    #Consistency Index (Lambda max - nb rows)/(nb_rows-1)
    ci = ((lambda_max-nb_rows)/(nb_rows-1))
    #print("Consistency Index =", ci)
    
    ##2.2-Random Index (RI)

    #The RI values, as a function of the matrix size, are given in (Saaty et al., 2007)
    ri = 0.0
    if nb_rows == 3:
        ri = 0.52
    elif nb_rows == 4:
        ri = 0.89
    elif nb_rows == 5:
        ri = 1.11
    elif nb_rows == 6:
        ri = 1.25
    elif nb_rows == 7:
        ri = 1.35
    elif nb_rows == 8:
        ri = 1.40
    elif nb_rows == 9:
        ri = 1.45
    elif nb_rows == 10:
        ri = 1.49
    elif nb_rows == 11:
        ri = 1.52
    elif nb_rows == 12:
        ri = 1.54
    elif nb_rows == 13:
        ri = 1.56
    elif nb_rows == 14:
        ri = 1.58
    elif nb_rows == 15:
        ri = 1.59
    #print("Random Index (RI) =", ri)

    ##2.3-Calculate Consistency Ratio (CR)
    cr = (ci/ri)
    #print("Consistency Ratio (CR) =",cr)
    
    #Create a csv file with the consistency index, the random index, and the consistency ratio
    filename = 'consistency_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    ciricr = (ci,ri,cr)
    np.savetxt(path_file, ciricr)
 

There are 10 paths in the list
Layer Weights: [0.3347 0.1452 0.0392 0.0321 0.1129 0.0297 0.0567 0.0859 0.1026 0.061 ]
Layer Weights: [0.2722 0.1804 0.0481 0.0129 0.0426 0.0201 0.1293 0.1419 0.0968 0.0557]


C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:28: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part


Layer Weights: [0.2085 0.1374 0.0274 0.0169 0.0521 0.0153 0.2379 0.2234 0.0261 0.055 ]
Layer Weights: [0.1686 0.0237 0.1317 0.0494 0.0457 0.0457 0.0494 0.2813 0.1466 0.058 ]
Layer Weights: [0.2287 0.1608 0.0156 0.0216 0.2581 0.0428 0.1024 0.0563 0.023  0.0906]
Layer Weights: [0.1424 0.0314 0.0437 0.3426 0.2748 0.0761 0.0595 0.0294]
Layer Weights: [0.0186 0.0723 0.0348 0.3331 0.0971 0.1636 0.056  0.2244]
Layer Weights: [0.017  0.0486 0.0292 0.3607 0.244  0.1498 0.0655 0.0853]
Layer Weights: [0.0366 0.3407 0.0871 0.2156 0.1071 0.0366 0.0924 0.0838]
Layer Weights: [0.0267 0.2781 0.0765 0.1557 0.0291 0.0517 0.1108 0.2713]


In [88]:
#### Compute the geometric mean of weights

from scipy.stats import gmean

def readmycsv(args):
    return pd.read_csv(args, header=None)

##Create a list per factor
land_cover = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_adults_lc*.csv'))
land_use = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_adults_lu*.csv'))

##Land cover
#Concatenate csv files and replace 0 by 1 (to avoid error when computing geometric mean)
weights_concat = pd.concat(map(readmycsv, land_cover), axis = 1, ignore_index=True)
weights_concat =  weights_concat.replace(0,1)
#Compute geometric mean, scale it and export to csv file
weights_gmean = weights_concat.apply(gmean, axis=1)
weights_gmean = ((weights_gmean - weights_gmean.min()) / (weights_gmean.max() - weights_gmean.min()))*100
weights_gmean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_adults_lc_geomean.csv'), header=None,index=False, float_format="%.0f")

##Land use
#Concatenate csv files and replace 0 by 1 (to avoid error when computing geometric mean)
weights_concat = pd.concat(map(readmycsv, land_use), axis = 1, ignore_index=True)
weights_concat =  weights_concat.replace(0,1)
#Compute geometric mean, scale it and export to csv file
weights_gmean = weights_concat.apply(gmean, axis=1)
weights_gmean = ((weights_gmean - weights_gmean.min()) / (weights_gmean.max() - weights_gmean.min()))*100
weights_gmean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_adults_lu_geomean.csv'), header=None,index=False, float_format="%.0f")


In [89]:
#### Produce the categorical factor layers

##LAND COVER

#Create a dataframe (df) with the recoding rules
mycsv = os.path.join(outdata['output_matrices_analysis'], 'weight_vectors_adults_lc_geomean.csv')
df_adults_lc_weights = pd.read_csv(mycsv, sep=';', header=None)
df_adults_lc_weights['lc_class'] = range(1, len(df_adults_lc_weights) + 1)
df_adults_lc_weights['equal'] = '='
df_adults_lc_weights = df_adults_lc_weights[['lc_class', 'equal', 0]]
#print (df_adults_lc_weights)

#Create a CSV file with the recoding rules (no header, no index, separator is space)
df_adults_lc_weights.to_csv(rules['classes_adults_lc'], header=False, index=False, sep = ' ')

#Reclassify the predictor layer
gscript.run_command('r.reclass', overwrite=True, input='LC_A', output='A_LC', rules=rules['classes_adults_lc'])


##LAND USE

#Create a dataframe (df) with the recoding rules
mycsv = os.path.join(outdata['output_matrices_analysis'], 'weight_vectors_adults_lu_geomean.csv')
df_adults_lu_weights = pd.read_csv(mycsv, sep=';', header=None)
df_adults_lu_weights['lu_class'] = range(1, len(df_adults_lu_weights) + 1)
df_adults_lu_weights['equal'] = '='
df_adults_lu_weights = df_adults_lu_weights[['lu_class', 'equal', 0]]
#print (df_adults_lu_weights)

#Create a CSV file with the recoding rules (no header, no index, separator is space)
df_adults_lu_weights.to_csv(rules['classes_adults_lu'], header=False, index=False, sep = ' ')

#Reclassify the predictor layer
gscript.run_command('r.reclass', overwrite=True, input='LU', output='A_LU', rules=rules['classes_adults_lu'])


0

### Membership function for scaling continuous factors

In [90]:
##DISPERSAL (SPREAD) FROM OPTIMAL BREEDING SITES

#Apply a function that reflects the dispersal range of adult Anopheles and normalise it to [0;100]
#The dispersal range 
gscript.run_command('r.mapcalc', overwrite=True,expression='A_DISPERSAL = if (L_HS_DIST == 0, 100, (if (L_HS_DIST > 0 && L_HS_DIST < 410, ((-0.0002505712*(L_HS_DIST^2)) + (-0.1403726401*(L_HS_DIST)) + 100), 0)))')

#Scale the values to [0;100] 
rastinfo = gscript.raster_info('A_DISPERSAL')
outmax = '%s_max' %('A_DISPERSAL')
rastmax = rastinfo['max']
print(outmax, '=', rastmax ) 
outmin = '%s_min' %('A_DISPERSAL')
rastmin = rastinfo['min']
print(outmin, '=', rastmin )

formula = '%s = round(((%s - %s) / (%s - %s)*100))' %('A_DISPERSAL','A_DISPERSAL',rastmin,rastmax,rastmin)
print(formula)
gscript.mapcalc(formula, overwrite=True)        

A_DISPERSAL_max = 100.0
A_DISPERSAL_min = 0.0
A_DISPERSAL = round(((A_DISPERSAL - 0.0) / (100.0 - 0.0)*100))


In [91]:
##DISTANCE TO BUILDINGS


#At a distance of 0m, we are in the buildings so suitability is maximal (100)
#Between 1 and 20m, suitability decreases with distance from 50 to 0
#After 100m, suitability is 0 again


#Calculate the slope and intercept of the linear function
#for rescaling the distance interval [1;20] to [50;0]
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress([1,20],[50,0])
print(slope,intercept)

#Calculate the factor value as a linear function of the distance, as y = ax + b (valid only for interval [10,100])
formula = '%s = (%s * %s) + %s' %('TEMP',slope,'DIST_BUILDINGS',intercept)
gscript.mapcalc(formula, overwrite=True)

#Calculate the factor value
gscript.run_command('r.mapcalc', overwrite=True,expression = "A_BU = round(if (DIST_BUILDINGS == 0, 100, if (DIST_BUILDINGS > 0 && DIST_BUILDINGS < 20, TEMP, 0)))")



-2.6315789473684212 52.631578947368425


0

In [92]:
## Print processing time
print_processing_time(starttime_scaling_factors,"Factors scaled in ")

'Factors scaled in 16.3 seconds'

## Weighting factors

In [93]:
##COMPARE THE RELATIVE IMPORTANCE OF THE FACTORS, AND THE ASSESS THE CONSISTENCY OF THE JUDGMENTS

#Comment on the warnings (from documentation): 
#When the matrix for which the eigenvalues and eigenvectors are computed
#is real, the resulting eigenvalues are real (0 imaginary part) or occur in conjugate pairs

#Create a list of csv files that contain the pairwise comparison matrices (PCM), and count the csv files
file_list = glob.glob(os.path.join(indata['matrices_adults_inter'], '*.csv'))
print("There are %s paths in the list"%len(file_list))

#Loop over the list of csv files
for file in file_list:
    #Read the pairwise comparison matrix (csv with headers) into df
    pcm_matrix = pd.read_csv(file, sep=';')   
    #print (pcm_matrix)

    #Convert into array
    pcm = np.array(pcm_matrix)
    #print (pcm)

    ##1-Calculate the priority vector (weight vector) of each PCM, i.e., its principal eigenvector
    
    eigenvalues, eigenvector=np.linalg.eig(pcm)
    maxindex=np.argmax(eigenvalues)
    eigenvalues=np.float32(eigenvalues) #float32
    eigenvector=np.float32(eigenvector) #float32
    weights=eigenvector[:, maxindex]

    weights = weights/np.sum(weights)

    #print ("Layer Weights: " + str(np.around(weights,4)))
    
    
    #Create a csv file with the weights
    filename = 'weight_vector_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    np.savetxt(path_file, weights, fmt='%1.4f')
    
    ##2-Calculate the consistency ratio (CR) of each PCM, 
    ##i.e., its consistency index (CI) versus the consistency index of a random-like array (RI) of same size
    ##CR=CI/RI
    ##CR<0.10 is acceptable (Saaty, 2012)

    ##2.1-Consistency Index (CI)

    #Multiply the pairwise comparison matrix by the weights, in column
    pcm_mult = np.multiply(pcm, weights)
    #print ("Matrix multiplied by weights (in column) =", pcm_mult)

    #Sum the lines
    weighted_sum = np.sum(pcm_mult, axis=1)
    #print ("Weighted sum (in line) =", weighted_sum)

    #Mean of (weighted sums divided by weights (priority))
    lambda_max = np.average(np.divide(weighted_sum, weights))
    #print("Lambda max =", lambda_max)

    #Count number of rows in pairwise comparison matrix
    nb_rows = pcm.shape[0]
    #print("Number of rows in the matrix =", nb_rows)

    #Consistency Index (Lambda max - nb rows)/(nb_rows-1)
    ci = ((lambda_max-nb_rows)/(nb_rows-1))
    #print("Consistency Index =", ci)
    
    ##2.2-Random Index (RI)

    #The RI values, as a function of the matrix size, are given in (Saaty et al., 2007)
    ri = 0.0
    if nb_rows == 3:
        ri = 0.52
    elif nb_rows == 4:
        ri = 0.89
    elif nb_rows == 5:
        ri = 1.11
    elif nb_rows == 6:
        ri = 1.25
    elif nb_rows == 7:
        ri = 1.35
    elif nb_rows == 8:
        ri = 1.40
    elif nb_rows == 9:
        ri = 1.45
    elif nb_rows == 10:
        ri = 1.49
    elif nb_rows == 11:
        ri = 1.52
    elif nb_rows == 12:
        ri = 1.54
    elif nb_rows == 13:
        ri = 1.56
    elif nb_rows == 14:
        ri = 1.58
    elif nb_rows == 15:
        ri = 1.59
    #print("Random Index (RI) =", ri)

    ##2.3-Consistency Ratio (CR)
    cr = (ci/ri)
    #print("Consistency Ratio (CR) =",cr)
    
    #Create a csv file with the consistency index, the random index, and the consistency ratio
    filename = 'consistency_%s' %(os.path.basename(file))
    path_file = os.path.join(outdata['output_matrices_analysis'], filename)
    ciricr = (ci,ri,cr)
    np.savetxt(path_file, ciricr)
    

There are 5 paths in the list


C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ADMIN_HOME\anaconda3\envs\mosquimap\lib\site-packages\ipykernel_launcher.py:26: ComplexWarning: Casting complex values to real discards the imaginary part


In [94]:
#### Compute arithmetic mean of weights

def readmycsv(args):
    return pd.read_csv(args, header=None)

##Create list of csv files
factors = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'weight_vector_adults_factors*.csv'))

#Concatenate csv files
weights_concat = pd.concat(map(readmycsv, factors), axis = 1, ignore_index=True)
#Compute arithmentic mean and export to csv file
weights_mean = weights_concat.mean(axis=1)
weights_mean.to_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_adults_factors_mean.csv'), header=None,index=False, float_format="%.4f")


In [95]:
#Create summary CSVs with consistency index, random index, and consistency ratio
#Ideally there should be row names and column names (to do when I have time)
#Row 1:Consistency Index; Row 2:Random Index; Row 3:Consistency Ratio

##Create lists
land_cover = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_adults_lc*.csv'))
land_use = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_adults_lu*.csv'))
factors = glob.glob(os.path.join(outdata['output_matrices_analysis'], 'consistency_adults_factors*.csv'))

#Concatenate land cover consistency
consistency_concat = pd.concat(map(readmycsv, land_cover), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'],"consistency_adults_concat_lc.csv"), index=False, header=None, float_format="%.4f")

#Concatenate land use consistency
#consistency_concat = pd.concat(map(readmycsv, land_use), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'], "consistency_adults_concat_lu.csv"), index=False, header=None, float_format="%.4f")

#Concatenate factors consistency
#consistency_concat = pd.concat(map(readmycsv, factors), axis = 1, ignore_index=True)
consistency_concat.to_csv(os.path.join(outdata['output_matrices_analysis'], "consistency_adults_concat_factors.csv"), index=False, header=None, float_format="%.4f")


####  <span style="color:purple">STOP ! Please check the consistency ratio and evaluate if experts should revise their judgments.</span>

In [53]:
input('Press enter to continue')

Press enter to continue 


' '

## Aggregating criteria to produce the adult Anopheles HSI map

In [96]:
# Start timer
starttime_hsi=start_processing()

In [97]:
#Set computational region
gscript.run_command('g.region', vector='AOI', align='DTM', res=5)

#Mask areas outside the population map
gscript.run_command('r.mask', overwrite=True, raster='POP')


0

In [98]:
#Breeding sites are defined as class "Optimal"
#Use the factor weights determined by vector ecology experts
#Calculate the weighted sum of factors to create the layer "ADULTS_SUITABILITY"

df_adults_factors_weights = pd.read_csv(os.path.join(outdata['output_matrices_analysis'],'weight_vectors_adults_factors_mean.csv'), header=None)
print(df_adults_factors_weights)
w1 = df_adults_factors_weights[0].values[0]
w2 = df_adults_factors_weights[0].values[1]
w3 = df_adults_factors_weights[0].values[2]
w4 = df_adults_factors_weights[0].values[3]

formula = '%s = (%s * %s) + (%s * %s) + (%s * %s) + (%s * %s)' %('ADULTS_SUITABILITY',w1, 'A_LC', w2, 'A_LU', w3, 'A_DISPERSAL', w4, 'A_BU')
gscript.mapcalc(formula, overwrite=True)


#Scale the values to [0;100] 
rastinfo = gscript.raster_info('ADULTS_SUITABILITY')
outmax = '%s_max' %('ADULTS_SUITABILITY')
rastmax = rastinfo['max']
print(outmax, '=', rastmax ) 
outmin = '%s_min' %('ADULTS_SUITABILITY')
rastmin = rastinfo['min']
print(outmin, '=', rastmin )

formula = '%s = round(((%s - %s) / (%s - %s)*100))' %('ADULTS_SUITABILITY','ADULTS_SUITABILITY',rastmin,rastmax,rastmin)
print(formula)
gscript.mapcalc(formula, overwrite=True)


        0
0  0.1075
1  0.1502
2  0.4700
3  0.2723
ADULTS_SUITABILITY_max = 100.0
ADULTS_SUITABILITY_min = 0.0
ADULTS_SUITABILITY = round(((ADULTS_SUITABILITY - 0.0) / (100.0 - 0.0)*100))


In [99]:
## Print processing time
print_processing_time(starttime_hsi,"HSI map produced in ")

'HSI map produced in 7.6 seconds'

## Resampling to 100m

In [100]:
# Start timer
starttime_resample=start_processing()

In [101]:
#Set computational region (use POP geometry)
gscript.run_command('g.region', raster='POP')

#Mask areas outside the population map
gscript.run_command('r.mask', overwrite=True, raster='POP')

0

In [102]:
#Resample HSI classes to 100 m (sum)
gscript.run_command('r.resamp.stats', overwrite=True, 
                    input='ADULTS_SUITABILITY', method='average', output='ADULTS_HSI_AVG_100M')

#Scale the values to [0;100] 
rastinfo = gscript.raster_info('ADULTS_HSI_AVG_100M')
outmax = '%s_max' %('ADULTS_HSI_AVG_100M')
rastmax = rastinfo['max']
print(outmax, '=', rastmax ) 
outmin = '%s_min' %('ADULTS_HSI_AVG_100M')
rastmin = rastinfo['min']
print(outmin, '=', rastmin )

formula = '%s = round(((%s - %s) / (%s - %s)*100))' %('ADULTS_HSI_AVG_100M','ADULTS_HSI_AVG_100M',rastmin,rastmax,rastmin)
print(formula)
gscript.mapcalc(formula, overwrite=True)



ADULTS_HSI_AVG_100M_max = 87.3325
ADULTS_HSI_AVG_100M_min = 0.0
ADULTS_HSI_AVG_100M = round(((ADULTS_HSI_AVG_100M - 0.0) / (87.3325 - 0.0)*100))


In [103]:
## Print processing time
print_processing_time(starttime_resample,"Resampling done in ")

'Resampling done in 2.6 seconds'

## Binning (e.g., 4 classes: Unsuitable, Marginal, Suitable and Optimal)

In [104]:
#Calculate percentiles
gscript.run_command('r.quantile', overwrite=True, input='ADULTS_HSI_AVG_100M', percentiles= [60, 85, 95], flags='r', file = rules['recode_adultshsi2classes'])
#Recode according to percentiles
gscript.run_command('r.recode', overwrite=True, input= 'ADULTS_HSI_AVG_100M', output='ADULTS_HS_CLASSES_100M', rules= rules['recode_adultshsi2classes'])
#Multiply by 10 (so the classes become 10, 20, 30, 40 and can be summed with the 3 population classes 1, 2, 3)
#to produce a bivariate map
formula = '%s = %s*10' %('ADULTS_HS_CLASSES_100M','ADULTS_HS_CLASSES_100M')
print(formula)
gscript.mapcalc(formula, overwrite=True)

ADULTS_HS_CLASSES_100M = ADULTS_HS_CLASSES_100M*10


In [105]:
#Export the result (hsi and hs classes at 100m) to tiff file, for making map layouts in QGIS
#gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='ADULTS_HSI_AVG_100M', output= os.path.join(outdata['outputdir'], 'dakar_adult_vectors_hsi_100m.tif'))
gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='ADULTS_HS_CLASSES_100M', output= os.path.join(outdata['outputdir'], 'dakar_adult_vectors_hs_classes_100m.tif'))

0

# Crossing adult HS and population density to produce a simple malaria exposure map


In [106]:
#Set computational region (use POP geometry)
gscript.run_command('g.region', raster='POP')

#Mask areas outside the population map
gscript.run_command('r.mask', overwrite=True, raster='POP')

#Combine adult Anopheles HS and population density (addition)
gscript.run_command('r.mapcalc', overwrite=True, 
                    expression='BIVARIATE_MALARIA_EXPOSURE = ADULTS_HS_CLASSES_100M + POP_LOG_CLASSES')



0

In [107]:
#Export map to tiff file, for making map layout in QGIS
gscript.run_command('r.out.gdal', overwrite=True, flags = 'm', input='BIVARIATE_MALARIA_EXPOSURE', output= os.path.join(outdata['outputdir'],'dakar_malaria_exposure_100m.tif'))

0

<span style="color:blue">Processing completed!</span>